In [3]:
!pip install biopython
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 20.3 MB/s eta 0:00:00


**Measuring Boyer-Moore's benefit**. First, download the Python module for Boyer-Moore preprocessing:



In [26]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py

--2025-06-15 17:02:35--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 18.65.227.179, 18.65.227.2, 18.65.227.145, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|18.65.227.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9400 (9.2K) [application/octet-stream]
Saving to: ‘bm_preproc.py.2’

bm_preproc.py.2     100%[===================>]   9.18K  --.-KB/s    in 0s      

2025-06-15 17:02:35 (27.1 MB/s) - ‘bm_preproc.py.2’ saved [9400/9400]



In [28]:
from google.colab import drive
drive.mount('/content')

ValueError: Mountpoint must not already contain files

In [27]:
import sys
sys.path.append('/content/')
import yfinance as yf
#!pip install --target=$nb_path bm_preproc.py

ERROR: Could not find a version that satisfies the requirement bm_preproc (from versions: none)
ERROR: No matching distribution found for bm_preproc


This module provides the BoyerMoore class, which encapsulates the preprocessing info used by the boyer_moore function above. Second, download the provided excerpt of human chromosome 1:

In [1]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2025-06-15 16:31:56--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 18.65.227.121, 18.65.227.179, 18.65.227.145, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|18.65.227.121|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  --.-KB/s    in 0.04s   

2025-06-15 16:31:56 (19.9 MB/s) - ‘chr1.GRCh38.excerpt.fasta’ saved [810105/810105]



In [5]:
human_chro = SeqIO.read("chr1.GRCh38.excerpt.fasta", "fasta").seq

print(human_chro[:20])
print(len(human_chro))

TTGAATGCTGAAATCAGCAG
800000


Third, implement versions of the naive exact matching and Boyer-Moore algorithms that additionally count and **return** \\
**(a)** the number of character comparisons performed \\


In [7]:
def naive_with_counts(pattern, text):
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    m = len(pattern)
    n = len(text)

    for i in range(n - m + 1):
        num_alignments += 1
        match = True
        for j in range(m):
            num_character_comparisons += 1
            if text[i + j] != pattern[j]:
                match = False
                break
        if match:
            occurrences.append(i)

    return occurrences, num_alignments, num_character_comparisons

**(b)** the number of alignments tried. Roughly speaking, these measure how much work the two different algorithms are doing.

In [8]:
def boyer_moore_with_counts(pattern, p_bm, text):
    """Boyer-Moore algorithm with counts for alignments and comparisons."""
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    m = len(pattern)
    n = len(text)
    i = 0  # Current alignment start in text

    while i <= n - m:
        num_alignments += 1
        shift = 1
        mismatch = False
        for j in range(m - 1, -1, -1):  # Compare from end to start
            num_character_comparisons += 1
            if text[i + j] != pattern[j]:
                # Bad character rule
                bc_shift = p_bm.bad_character_rule(j, text[i + j])
                # Good suffix rule
                gs_shift = p_bm.good_suffix_rule(j)
                shift = max(bc_shift, gs_shift)
                mismatch = True
                break
        if not mismatch:
            occurrences.append(i)
            gs_shift = p_bm.match_skip()
            shift = max(shift, gs_shift)
        i += shift

    return occurrences, num_alignments, num_character_comparisons

Question 1: How many alignments does the naive exact matching algorithm try when matching the string
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [10]:
naive_with_counts("GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG", human_chro)

([56922], 799954, 984143)

Question 2: How many character comparisons does the naive exact matching algorithm try when matching the string
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [11]:
naive_with_counts("GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG", human_chro)

([56922], 799954, 984143)

Question 3: How many alignments does Boyer-Moore try when matching the string
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [30]:
from bm_preproc import BoyerMoore

alu = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(alu, lowercase_alphabet)

boyer_moore_with_counts(alu, p_bm, human_chro)

AssertionError: 